In [187]:
import pandas as pd
import numpy as np

In [188]:
# Define the index
index = ['A', 'B', 'C', 'D']

# Define the data for the columns
data = {
    'Q': [None, None, None, None],  
    'deltaP': [None, None, None, None],  
    'Power': [None, None, None, None],  
    'Efficiency': [None, None, None, None],
    'Power input correction' : [None, None, None, None] 
}

# Create the DataFrame
PUMP = pd.DataFrame(data, index=index)


# Define the data for the columns
data_OFF = {
    'Q': [None, None, None, None],  
    'deltaP': [None, None, None, None],  
    'Power': [None, None, None, None],  
    'Efficiency': [None, None, None, None] ,
    'Power input correction' : [None, None, None, None] 
}

# Create the DataFrame
PUMP_OFF = pd.DataFrame(data_OFF, index=index)



EN14825:2022(E) PG 65 

The SEPR is defined as the reference annual process cooling demand divided by the energy consumption, calculated according formula bellow:

$$
SEPR = \frac{\sum (C_i \cdot h_i)}{E_{total}}
$$

Where:

- 𝐶𝑖 is the cooling capacity at part load condition 𝑖
- ℎ𝑖 is the operating hours at part load condition 𝑖
- 𝐸𝑡𝑜𝑡𝑎𝑙 is the total electrical energy consumption over the defined period

Expanding:

$$
SEPR = \frac{\sum_{j=1}^{n} \left( Pc(Tj) \cdot h_j \right)}{\sum_{j=1}^{n} \left( \frac{Pc(Tj) \cdot h_j}{EERbin(Tj)} \right)}
$$

Where:

- Tj is the bin temperature
- j is the bin number
- n is the total number of bins
- Pc(Tj) is the process cooling load for the corrisponding bin temperature Tj
- hj is the number of bin hours occuring at the corresponding bin temperature
- EERbin(Tj) is the EER value of the unit for the corrisponding bin temperature Tj

Parameters INPUT:

In [189]:
Pdesign = 55000  # Chiller cooling capacity in W - INPUT

In [190]:
#TEST A - Tamb 35°, Water inlet 12°C, water outlet 7°C

Pdc_A = Pdesign # Cooling capacity at 35 °C - [W] - INPUT
Q_A = 20 # Pump flow rate at conditions A in [m3/s] - INPUT
Delta_Pe_A = 600 #External static pressure difference between pump outlet and inlet [Pa] - INPUT
Pinput_A = 60000 # Power input at 35 °C - [W] - INPUT
Q_A_OFF = 20 # Pump flow rate at conditions A in [m3/s] - INPUT - OPTIONAL
Delta_Pe_A_OFF = 600 #External static pressure difference between pump outlet and inlet [Pa] - INPUT - OPTIONAL
Pinput_A_OFF = 20000 # Power input at 35 °C - [W] - INPUT

In [191]:
#TEST B - Tamb 25°,  water outlet 7°C, SAME FLOWRATE

Pdc_B = 53000 # Cooling capacity at 25 °C - Value to be retrieved at testing room - [W] - INPUT
Q_B = 20 # Pump flow rate at conditions B in [m3/s] - INPUT
Delta_Pe_B = 600 #External static pressure difference between pump outlet and inlet [Pa] - INPUT
Pinput_B = 57000# Power input at 25 °C - Value to be retrieved at testing room - [W] - INPUT
Q_B_OFF = Q_A_OFF # Pump flow rate at conditions B in [m3/s] - INPUT
Delta_Pe_B_OFF = Delta_Pe_A_OFF #External static pressure difference between pump outlet and inlet [Pa] - INPUT
Pinput_B_OFF = Pinput_A_OFF # Power input at 25 °C - Value to be retrieved at testing room - [W] - INPUT

In [192]:
#TEST C - Tamb 15°,  water outlet 7°C, SAME FLOWRATE

Pdc_C = 53000 # Cooling capacity at 15 °C - Value to be retrieved at testing room - [W] - INPUT
Q_C = 20 # Pump flow rate at conditions B in [m3/s] - INPUT
Delta_Pe_C = 600 #External static pressure difference between pump outlet and inlet [Pa] - INPUT
Pinput_C = 50000# Power input at 15 °C - Value to be retrieved at testing room - [W] - INPUT
Q_C_OFF = Q_A_OFF # Pump flow rate at conditions B in [m3/s] - INPUT
Delta_Pe_C_OFF = Delta_Pe_A_OFF #External static pressure difference between pump outlet and inlet [Pa] - INPUT
Pinput_C_OFF = Pinput_A_OFF # Power input at 15 °C - Value to be retrieved at testing room - [W] - INPUT

In [193]:
#TEST D - Tamb 5°,  water outlet 7°C, SAME FLOWRATE

Pdc_D = 53000 # Cooling capacity at 5 °C - Value to be retrieved at testing room - [W] - INPUT
Q_D = 20 # Pump flow rate at conditions B in [m3/s] - INPUT
Delta_Pe_D = 600 #External static pressure difference between pump outlet and inlet [Pa] - INPUT
Pinput_D = 42000# Power input at 5 °C - Value to be retrieved at testing room - [W] - INPUT
Q_D_OFF = Q_A_OFF # Pump flow rate at conditions B in [m3/s] - INPUT
Delta_Pe_D_OFF = Delta_Pe_A_OFF #External static pressure difference between pump outlet and inlet [Pa] - INPUT
Pinput_D_OFF = Pinput_A_OFF# Power input at 5 °C - Value to be retrieved at testing room - [W] - INPUT

Tj is the bin temperature while hj is the number of bin hours occurring at the corresponding, defined by EN14825:2022 :

In [194]:
# Air cooled process chillers, annex C, page 85

Tj = list(range(-19,39)) 
hj = [0.08, 0.41, 0.65, 1.05, 1.74, 2.98, 3.79, 5.69, 8.94, 11.81, 17.29, 20.02, 28.73, 39.71, 56.61, 76.36, 106.07, 153.22, 203.41, 247.98, 
      282.01, 275.91, 300.61, 310.77, 336.48, 350.48, 363.49, 368.91, 371.63, 377.32, 376.53, 386.42, 389.84, 384.45, 370.45, 344.96, 328.02,
      305.36, 261.87, 223.9, 196.31, 163.04, 141.78, 121.93, 104.46, 85.77, 71.54, 56.57, 43.35, 31.02, 20.21, 11.85, 8.17, 3.83, 2.09, 1.21, 
      0.52, 0.4]

Pc(Tj) is the process cooling load at a given temperature and is defined by multiplying the full-load value (Pdesig) with part load ratio for each bin. See formula 34, page 66 EN14825:22:

$$
Pc(Tj) = Pdesign \cdot Pr(Tj)
$$

In [195]:
PrTj = []  # Part load ratio list

for value in Tj:
    pr_value = 0.8 + 0.2 * (value - 5) / (35 - 5)  # Calculate the part load for each bin

    # Apply the condition: if value is smaller than 0.8, set it to 0.8; if it's larger than 1, set it to 1
    if pr_value < 0.8:
        pr_value = 0.8
    elif pr_value > 1:
        pr_value = 1

    PrTj.append(pr_value)  # Append the calculated value to the list

PcTj = []  # Process cooling load list

for value in PrTj:
    pc_value = Pdesign * value  # Multiply Pdesign by the current part load ratio value
    PcTj.append(pc_value)  # Append the calculated value to the list

Assign values retrieved to the storage dataframe:

In [196]:
# Create the DataFrame
df = pd.DataFrame({
    'Tj': Tj,
    'hj': hj,
    'PrTj': PrTj,
    'PcTj': PcTj,  # PcTj (cooling load) column with empty values (None)
    'Pdc':[None] * len(Tj),  # Unity capacity at Tj temperature (COOLING DEMAND - experimental)
    'EER': [None] * len(Tj),  # EER column with empty values (None)
    'EER Bin':[None] * len(Tj),  # EER column with empty values (None) 
    'EER Bin CD':[None] * len(Tj),  # EER column with empty values (None)
    'CR':[None] * len(Tj),  # CR column with empty values (None)
    'SEPR1':[None] * len(Tj),  # SEPR column with empty values
    'SEPR2':[None] * len(Tj),  # SEPR column with empty values
    'SEPR2 CD':[None] * len(Tj),  # SEPR column with empty values
})

# Assign values ;

df.loc[54, 'Pdc'] = Pdc_A
df.loc[44, 'Pdc'] = Pdc_B
df.loc[34, 'Pdc'] = Pdc_C
df.loc[24, 'Pdc'] = Pdc_D

# Replace None values with NaN in the entire DataFrame
df = df.replace({None: np.nan})


C:\Users\e2023898\AppData\Local\Temp\ipykernel_19704\801232350.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({None: np.nan})


Regarding the power input, in what relates to the fan, in the case of units that are not designed for duct conncetion and is equipped with an integral fan, the power absorbed by the fan shall be included in the effective power absorbed by the unit (EN 14511-3:2022). In the case of the integrated pump, only a fraction of the input to pump motor shall be included in the total power absorbed by the unit. So, a correction factor to be subtracted of the total power abosorbed by the unit must be performed as follow:


note: Same procedure must be repeated with compressor turned OFF, after 10 min during 5 minutes

The function bellow define the efficiency of dry motor pumps with power smaller than 500W.

$$
\eta = 0.0721 \cdot P_{hyd}^{0.3183}
$$

<ul>
  <li>&eta; is the pump efficiency</li>
  <li>P<sub>hyd</sub> is the pump power</li>
</ul>

or for pumps with power > 500W
$$
\eta = 0.092 \cdot \ln(P_{\text{hyd}}) - 0.0403
$$

where:

<ul>
  <li>&eta; is the pump efficiency</li>
  <li>P<sub>hyd</sub> is the pump power</li>
</ul>



In [197]:
# Define the function to calculate efficiency based on Power
def calculate_efficiency(row):
    power = row['Power']
    if power <= 500:
        # Efficiency formula when Power is greater than 1
        return 0.0721 * (power ** 0.3183)
    else:
        # Efficiency formula when Power is smaller than or equal to 1
        return 0.092 * np.log(power) - 0.0403

Fraction of the pump power to be excluded from unit power absorbed is calculated bellow:
$$
P_{\text{hyd}} = \frac{q \cdot \Delta P_{e}}{\eta}
$$

Where:
<ul>
  <li>P<sub>hyd</sub> is the pump power </li>
  <li>&Delta;P<sub>e</sub> is the measure available external static pressure in Pa (EN14511-3:2018)</li>
  <li>&eta; is the pump efficiency</li>
</ul>



In [198]:
# Power input correction EN14511-3:2018(E)

P_hyd_A = Q_A * Delta_Pe_A # Pump idraulic power [W]
P_hyd_B = Q_B * Delta_Pe_B # Pump idraulic power [W]
P_hyd_C = Q_C * Delta_Pe_C # Pump idraulic power [W]
P_hyd_D = Q_D * Delta_Pe_D # Pump idraulic power [W]

# Define the index
index = ['A', 'B', 'C', 'D']

# Define the data for the columns
data = {
    'Q': [Q_A, Q_B, Q_C, Q_D],  
    'deltaP': [Delta_Pe_A, Delta_Pe_B, Delta_Pe_C, Delta_Pe_D],  
    'Power': [P_hyd_A, P_hyd_B, P_hyd_C, P_hyd_D],  
    'Efficiency': [None, None, None, None] ,
    'Power input correction' : [None, None, None, None] 
}

# Create the DataFrame
PUMP = pd.DataFrame(data, index=index)

# Apply the function to the 'Efficiency' column
PUMP['Efficiency'] = PUMP.apply(calculate_efficiency, axis=1)

# Calculate power input correction factor 'Power input correction'
PUMP['Power input correction'] = PUMP['Q'] * PUMP['deltaP'] / PUMP['Efficiency']

PUMP.head()

,Q,deltaP,Power,Efficiency,Power input correction
A,20,600,12000,0.823825,14566.202159
B,20,600,12000,0.823825,14566.202159
C,20,600,12000,0.823825,14566.202159
D,20,600,12000,0.823825,14566.202159


In [199]:
# Power input correction EN14511-3:2018(E)

P_hyd_A_OFF = Q_A_OFF * Delta_Pe_A_OFF # Pump idraulic power [W]
P_hyd_B_OFF = Q_B_OFF * Delta_Pe_B_OFF # Pump idraulic power [W]
P_hyd_C_OFF = Q_C_OFF * Delta_Pe_C_OFF # Pump idraulic power [W]
P_hyd_D_OFF = Q_D_OFF * Delta_Pe_D_OFF # Pump idraulic power [W]

# Define the data for the columns
data_OFF = {
    'Q': [Q_A_OFF, Q_B_OFF, Q_C_OFF, Q_D_OFF],  
    'deltaP': [Delta_Pe_A_OFF, Delta_Pe_B_OFF, Delta_Pe_C_OFF, Delta_Pe_D_OFF],  
    'Power': [P_hyd_A_OFF, P_hyd_B_OFF, P_hyd_C_OFF, P_hyd_D_OFF],  
    'Efficiency': [None, None, None, None] ,
    'Power input correction' : [None, None, None, None] 
}

# Create the DataFrame
PUMP_OFF = pd.DataFrame(data_OFF, index=index)

# Apply the function to the 'Efficiency' column
PUMP_OFF['Efficiency'] = PUMP_OFF.apply(calculate_efficiency, axis=1)

# Calculate power input correction factor 'Power input correction'
PUMP_OFF['Power input correction'] = PUMP_OFF['Q'] * PUMP_OFF['deltaP'] / PUMP_OFF['Efficiency']

PUMP_OFF.head()


,Q,deltaP,Power,Efficiency,Power input correction
A,20,600,12000,0.823825,14566.202159
B,20,600,12000,0.823825,14566.202159
C,20,600,12000,0.823825,14566.202159
D,20,600,12000,0.823825,14566.202159


In [200]:
# Corrected power input follow

Pinput_A_corrected = Pinput_A - PUMP.iloc[0,-1]
Pinput_B_corrected = Pinput_B - PUMP.iloc[1,-1]
Pinput_C_corrected = Pinput_C - PUMP.iloc[2,-1]
Pinput_D_corrected = Pinput_D - PUMP.iloc[3,-1]

Pinput_A_OFF_corrected = Pinput_A_OFF - PUMP_OFF.iloc[0,-1]
Pinput_B_OFF_corrected = Pinput_B_OFF - PUMP_OFF.iloc[1,-1]
Pinput_C_OFF_corrected = Pinput_C_OFF - PUMP_OFF.iloc[2,-1]
Pinput_D_OFF_corrected = Pinput_D_OFF - PUMP_OFF.iloc[3,-1]

Calculation of the degradation coefficient considering according EN14825:2022 section 11.5.3 pg 73 AND en14511-3:2018.
To calculate CD, for each part load test A, B, C and D the electrical power input during the compressor-off state  of the unit is measured during 5 min after the compressor has been switched off for 10 min after the end of the part load test for which the CD shall be determined.

$$
C{\text{d}} = 1-\frac{P{\text{coff}}}{P{\text{con}}}
$$

Where:
<ul>
  <li>C<sub>d</sub> is the degradation coefficient </li>
  <li>P<sub>coff</sub> is the power input during compressor-off state </li>
  <li>P<sub>con</sub> is the power input during corresponding part-load test </li>
</ul>


In [201]:
# Creating the DataFrame
data = {
    'Corrected power ON': [Pinput_A_corrected, Pinput_B_corrected, Pinput_C_corrected, Pinput_D_corrected],
    'Corrected power OFF': [Pinput_A_OFF_corrected, Pinput_B_OFF_corrected, Pinput_C_OFF_corrected, Pinput_D_OFF_corrected]
}

CD_TOT = pd.DataFrame(data)

# Calculating 'CD' column
CD_TOT['CD'] = 1 - (CD_TOT['Corrected power OFF'] / CD_TOT['Corrected power ON'])

CD_TOT.head()


,Corrected power ON,Corrected power OFF,CD
0,45433.797841,5433.797841,0.880402
1,42433.797841,5433.797841,0.871946
2,35433.797841,5433.797841,0.846649
3,27433.797841,5433.797841,0.801931


Calculation of the capacity ratio of the compressor according formula 36 of EN14825:2022 PG 67:
$$
C_{\text{R}} = \frac{P_{r}(T_{j}) \cdot P_{\text{design}}}{P_{\text{dc}}}
$$

Where:
<ul>
  <li>C<sub>r</sub> is the capacity ratio </li>
  <li>P<sub>r</sub>T<sub>j</sub> is the part load ratio </li>
  <li>P<sub>design</sub> is the full load value </li>
  <li>P<sub>dc</sub> is the declared capacity of the unit at conditions B, C and D </li>
</ul>

In [202]:
# Calculating  capacity ratio for  points B, C and D

# Calculate and update CR values with capping
df.loc[54, 'CR'] = df.loc[54, 'PrTj'] * Pdesign / df.loc[54, 'Pdc']
if df.loc[54, 'CR'] > 1:
    df.loc[54, 'CR'] = 1

df.loc[44, 'CR'] = df.loc[44, 'PrTj'] * Pdesign / df.loc[44, 'Pdc']
if df.loc[44, 'CR'] > 1:
    df.loc[44, 'CR'] = 1

df.loc[34, 'CR'] = df.loc[34, 'PrTj'] * Pdesign / df.loc[34, 'Pdc']
if df.loc[34, 'CR'] > 1:
    df.loc[34, 'CR'] = 1

df.loc[24, 'CR'] = df.loc[24, 'PrTj'] * Pdesign / df.loc[24, 'Pdc']
if df.loc[24, 'CR'] > 1:
    df.loc[24, 'CR'] = 1

The EER is the declared cooling capacity of the unit divided by the effective power input of the unit at specic temperature conditions. The EER (Energy efficiency ratio) values at each bin are determined via linear interpolation of the ERRbin values at part load conditions A, B, C and D (See table 17 section 9 EN14825:2022 PAGE 62 for water cooled process chiller and tabe 16 for air cooled process chillers).  For part load condition A the declared capacity of a unity is considered equal to the process cooling load (Pdesign). As in TEXA most of Chillers do not use variable speed compressors, we going throungh the second cas described in the standard. If the declared capacity of a unit is higher than the process cooling load by  more than 3%, the unit has to cycle ON/OFF and a decradation coefficient Cd nas to be used to calculate the ERR.
$$
\text{EER}_d = \frac{P_{\text{dc}}}{P_{\text{input-corrected}}}
$$

where:
<ul>
  <li>P<sub>dc</sub> is the declared capacity of the unit at conditions B, C and D </li>
  <li>P<sub>input-corrected</sub> is the electrical power input corrected for points B, C and D </li>
</ul>


In [203]:
# EER_d calculation according section 3.1.37 of UNI14825:2022(E) pg17

df.loc[54, 'EER'] = Pdc_A/Pinput_A_corrected
df.loc[44, 'EER'] = Pdc_B/Pinput_B_corrected
df.loc[34, 'EER'] = Pdc_C/Pinput_C_corrected
df.loc[24, 'EER'] = Pdc_D/Pinput_D_corrected


# Now interpolate the 'EER' column based on index
#df['EER'] = df['EER'].interpolate(method='index')

# Update the EER column based on the conditions
#df.loc[df['PrTj'] <= 0.8, 'EER'] = df.loc[24, 'EER']  # Set EER to the value at PrTj = 0.8 for rows where PrTj < 0.8
#df.loc[df['PrTj'] >= 1, 'EER'] = df.loc[54, 'EER']  # Set EER to the value at PrTj = 1 for rows where PrTj > 1

In [204]:
# Degradation coefficient

# According section 10.2 of EN14825:2022 for non determined CD a default values of 0.9 must be considered

CD = 0.9 # - INPUT

coefficients = [54, 44, 34, 24]

for index in df.index:
    if index in coefficients:
        # Perform the calculation for each row with specified coefficients and assign the result to 'EER Bin'
        df.loc[index, 'EER Bin'] = df.loc[index, 'EER'] * df.loc[index, 'CR'] / (CD * df.loc[index, 'CR'] + (1 - CD))


# Now interpolate the 'EER Bin' column based on index
df['EER Bin'] = df['EER Bin'].interpolate(method='index')


# Update the EER column based on the conditions
df.loc[df['PrTj'] <= 0.8, 'EER Bin'] = df.loc[24, 'EER Bin']  # Set EER to the value at PrTj = 0.8 for rows where PrTj < 0.8
df.loc[df['PrTj'] >= 1, 'EER Bin'] = df.loc[54, 'EER Bin']  # Set EER to the value at PrTj = 1 for rows where PrTj > 1

In [205]:
# Same calculation using the CD values measured and calculated:

coefficients = [54, 44, 34, 24]
cd_values = CD_TOT['CD'].iloc[:4]  # Assuming we are using the first 4 values from CD_TOT['CD']

for idx, coef in enumerate(coefficients):
    if coef in df.index:
        # Perform the calculation for each row with specified coefficients and assign the result to 'EER Bin CD'
        CD = cd_values.iloc[idx]
        df.loc[coef, 'EER Bin CD'] = df.loc[coef, 'EER'] * df.loc[coef, 'CR'] / (CD * df.loc[coef, 'CR'] + (1 - CD)) 

# Now interpolate the 'EER Bin' column based on index
df['EER Bin CD'] = df['EER Bin CD'].interpolate(method='index')


# Update the EER column based on the conditions
df.loc[df['PrTj'] <= 0.8, 'EER Bin CD'] = df.loc[24, 'EER Bin CD']  # Set EER to the value at PrTj = 0.8 for rows where PrTj < 0.8
df.loc[df['PrTj'] >= 1, 'EER Bin CD'] = df.loc[54, 'EER Bin CD']  # Set EER to the value at PrTj = 1 for rows where PrTj > 1 

Calculation of the partials needed for SEPR calculation: 

In [207]:
# Iterate over all indexes of the DataFrame and apply the formula
for index in df.index:
    # Calculate SEPR using the formula
    df.loc[index, 'SEPR1'] = df.loc[index, 'hj'] * df.loc[index, 'PcTj']
    df.loc[index, 'SEPR2'] = df.loc[index, 'hj'] * (df.loc[index, 'PcTj'] / (df.loc[index, 'EER Bin']))
    df.loc[index, 'SEPR2 CD'] = df.loc[index, 'hj'] * (df.loc[index, 'PcTj'] / (df.loc[index, 'EER Bin CD']))

Results are displayed bellow:

In [218]:
# Sum all values in the 'SEPR' column
total_sepr1 = df['SEPR1'].sum()
total_sepr2 = df['SEPR2'].sum()
total_sepr2_CD = df['SEPR2 CD'].sum()

SEPR = total_sepr1/total_sepr2
SEPR_CD = total_sepr1/total_sepr2_CD

# Print the total sum
#print("Total sum of 1st partial:", total_sepr1)
#print("Total sum of 2nd partial:", total_sepr2)
print("SEPR, considering CD = 0.9: {:.3f}".format(SEPR))
print("SEPR, considering calculated CD: {:.3f}".format(SEPR_CD))



Total sum of SEPR values, considering CD = 0.9: 1.597
Total sum of SEPR values, considering calculated CD: 1.580


In [222]:
# Define the indexes and columns
indexes = ['A', 'B', 'C', 'D']
columns = ['EER theorical', 'Cooling capacity (W)', 'CD', 'CR', 'Corrected EER', 'Corrected EER CD = 0.9']

# Create the DataFrame
overview = pd.DataFrame(index=indexes, columns=columns)

# Populate the DataFrame with values from df and CD_TOT
overview.loc['A'] = [df.loc[54, 'EER'], df.loc[54, 'Pdc'], CD_TOT.loc[0, 'CD'], df.loc[54, 'CR'], df.loc[54, 'EER Bin CD'], df.loc[54, 'EER Bin']]
overview.loc['B'] = [df.loc[44, 'EER'], df.loc[44, 'Pdc'], CD_TOT.loc[1, 'CD'], df.loc[44, 'CR'], df.loc[44, 'EER Bin CD'], df.loc[44, 'EER Bin']]
overview.loc['C'] = [df.loc[34, 'EER'], df.loc[34, 'Pdc'], CD_TOT.loc[2, 'CD'], df.loc[34, 'CR'], df.loc[34, 'EER Bin CD'], df.loc[34, 'EER Bin']]
overview.loc['D'] = [df.loc[24, 'EER'], df.loc[24, 'Pdc'], CD_TOT.loc[3, 'CD'], df.loc[24, 'CR'], df.loc[24, 'EER Bin CD'], df.loc[24, 'EER Bin']]

# Format the values to three decimal places
overview = overview.applymap(lambda x: f"{x:.3f}")

# Style the DataFrame
overview_styled = overview.style.set_properties(**{
    'text-align': 'center'
}).set_table_styles([
    {'selector': 'th', 'props': [('font-weight', 'bold'), ('text-align', 'center')]}
])

# Display the styled DataFrame
overview_styled


C:\Users\e2023898\AppData\Local\Temp\ipykernel_19704\3813061403.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  overview = overview.applymap(lambda x: f"{x:.3f}")


,EER theorical,Cooling capacity (W),CD,CR,Corrected EER,Corrected EER CD = 0.9
A,1.211,55000.000,0.880,1.000,1.211,1.211
B,1.249,53000.000,0.872,0.969,1.244,1.245
C,1.496,53000.000,0.847,0.899,1.471,1.479
D,1.932,53000.000,0.802,0.830,1.857,1.893


In [215]:
columns = ['Bin temperature (Tj) [°C]', 'Bin hours (hj) [h]', 'Part load (PrTj) [%]', 
           'Cooling demand (PcTj) [W]', 'Efficiency ratio (EERbin)', 'Efficiency ratio (EERbin CD)']

results = pd.DataFrame({
    'Bin temperature (Tj) [°C]': df['Tj'],
    'Bin hours (hj)  [h]': df['hj'],
    'Part load (PrTj) [%]': df['PrTj'],
    'Cooling demand (PcTj) [W]': df['PcTj'],
    'Efficiency ratio (EERbin)': df['EER Bin'],
    'Efficiency ratio (EERbin CD)': df['EER Bin CD']
})

# Set 'j' as the index and start from 1
results.index = np.arange(1, len(results) + 1)

# Function to highlight specified rows in light blue and make letters bold black
def highlight_rows(x):
    color = 'background-color: lightblue'
    bold_black = 'font-weight: bold; color: black'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.loc[[54, 44, 34, 24], :] = color
    df1.loc[[54, 44, 34, 24], :] = bold_black
    return df1

# Apply the highlighting function
results_styled = results.style.apply(highlight_rows, axis=None)

# Remove index title, bold the titles, center values, and format decimal places
results_styled = results_styled.set_table_styles([{
    'selector': 'th',
    'props': [('font-weight', 'bold'), ('text-align', 'center')]
}, {
    'selector': 'td',
    'props': [('text-align', 'center')]
}])

# Formatting columns
results_styled = results_styled.format({
    'Bin temperature (Tj) [°C]': '{:.0f}',
    'Bin hours (hj)  [h]': '{:.3f}',
    'Part load (PrTj) [%]': '{:.3f}',
    'Cooling demand (PcTj) [W]': '{:.3f}',
    'Efficiency ratio (EERbin)': '{:.3f}',
    'Efficiency ratio (EERbin CD)': '{:.3f}'
})

# Display the styled DataFrame
results_styled


,Bin temperature (Tj) [°C],Bin hours (hj) [h],Part load (PrTj) [%],Cooling demand (PcTj) [W],Efficiency ratio (EERbin),Efficiency ratio (EERbin CD)
1,-19,0.080,0.800,44000.000,1.893,1.857
2,-18,0.410,0.800,44000.000,1.893,1.857
3,-17,0.650,0.800,44000.000,1.893,1.857
4,-16,1.050,0.800,44000.000,1.893,1.857
5,-15,1.740,0.800,44000.000,1.893,1.857
6,-14,2.980,0.800,44000.000,1.893,1.857
7,-13,3.790,0.800,44000.000,1.893,1.857
8,-12,5.690,0.800,44000.000,1.893,1.857
9,-11,8.940,0.800,44000.000,1.893,1.857
10,-10,11.810,0.800,44000.000,1.893,1.857
